In [1]:
import altair as alt
import matplotlib

matplotlib.style.use("ggplot")
import arrow
from IPython.display import display
import pandas_datareader.data as data


%matplotlib inline

ModuleNotFoundError: No module named 'pandas_datareader'

# Lets take a stab at some stock analysis!

In [ ]:
start_date = "2017-06-01"  # SNAP  IPO'd on 3/2/2017 - so start after that.
end_date = "2020-12-31"

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = data.DataReader(
    "FB;AMZN;SNAP;AAPL;GOOG;MSFT;GOLD".split(";"), "yahoo", start_date, end_date
)
print("Sample from the full dataset")
display(panel_data.head(1))
df_original = panel_data["Close"]
print("Sample of data from close")
df = df_original.copy()
display(df.head(3))

In [22]:
earliest = arrow.utcnow().shift(months=-24).date()
df = df_original.copy()[
    earliest:
]  # note this is destructive, probably good to keep an original around as a best practice

df.index = df.index.astype(
    str
)  # when transposing dates to columns, easier to operate in strings.
display(df)

first_day = df.reset_index().iloc[0, 0]
last_day = df.reset_index().iloc[-1, 0]

display(f"Returns from:{first_day}, to:{last_day}")

returns = df.iloc[
    [0, -1]
].T  # first and last row, and turn into columns for easy manipulation
returns["delta"] = returns[last_day] - returns[first_day]
returns["pcnt_change"] = returns.delta / returns[first_day]
display(returns)

# returns.diff = returns.[] _[0] - _[1]
print(
    "QQ: Should sum daily %% change ==  pcnt_change of total returns -- because it doesn't??"
)
df.pct_change().sum()

Symbols,FB,AMZN,SNAP,AAPL,GOOG,MSFT,GOLD
Date,,,,,,,
2020-04-03,154.179993,1906.589966,11.060000,60.352501,1097.880005,153.830002,19.950001
2020-04-06,165.550003,1997.589966,12.160000,65.617500,1186.920044,165.270004,20.549999
2020-04-07,168.830002,2011.599976,12.100000,64.857498,1186.510010,163.490005,20.309999
2020-04-08,174.279999,2043.000000,13.220000,66.517502,1210.280029,165.130005,20.410000
2020-04-09,175.190002,2042.760010,13.610000,66.997498,1211.449951,165.139999,22.510000
...,...,...,...,...,...,...,...
2020-12-24,267.399994,3172.689941,50.150002,131.970001,1738.849976,222.750000,22.900000
2020-12-28,277.000000,3283.959961,48.259998,136.690002,1776.089966,224.960007,22.840000
2020-12-29,276.779999,3322.000000,51.230000,134.869995,1758.719971,224.149994,22.910000


'Returns from:2020-04-03, to:2020-12-31'

Date,2020-04-03,2020-12-31,delta,pcnt_change
Symbols,,,,
FB,154.179993,273.160004,118.980011,0.771696
AMZN,1906.589966,3256.929932,1350.339966,0.708249
SNAP,11.060000,50.070000,39.009999,3.527125
AAPL,60.352501,132.690002,72.337502,1.198583
GOOG,1097.880005,1751.880005,654.000000,0.595694
MSFT,153.830002,222.419998,68.589996,0.445882
GOLD,19.950001,22.780001,2.830000,0.141855


QQ: Should sum daily %% change ==  pcnt_change of total returns -- because it doesn't??


Symbols
FB      0.631636
AMZN    0.584553
SNAP    1.703472
AAPL    0.842155
GOOG    0.503382
MSFT    0.405675
GOLD    0.199222
dtype: float64

In [23]:
print("Correlations between stocks")
print("  NOTE: Need to correlate on percentage change, not abosolute price")


def display_correlation_matrix(df, earliest, latest):
    df = df_original.copy()
    df.columns.name = None
    corr = df.pct_change(1).corr()  # compute correlation on percent change
    df_dates = df[earliest:latest]
    corr = df_dates.corr().reset_index().melt(id_vars="index")
    # display(corr)
    height_in_inches = 40

    base = (
        alt.Chart(corr)
        .properties(
            width=8 * height_in_inches,
            height=8 * height_in_inches,
            title=f"Stock Correlation from: {earliest} to {latest}",
        )
        .encode(
            x="index:O",
            y="variable:O",
        )
    )

    c = base.mark_rect().encode(
        tooltip="value;index;variable".split(";"), color="value:Q"
    )
    text = base.mark_text(baseline="middle").encode(
        text=alt.Text("value:Q", format="0.2f")
    )
    display(c + text)


# Look at correlation
year_ago = arrow.utcnow().shift(months=-12).date()
today = arrow.utcnow().date()
latest = today
cv_19_start = "2020-02-01"


first_day = df_original.reset_index().iloc[0, 0]

display_correlation_matrix(df, year_ago, today)
display_correlation_matrix(df, first_day, today)
display_correlation_matrix(df, "2020-06-01", today)
display_correlation_matrix(df, "2019-01-01", cv_19_start)
display_correlation_matrix(df, cv_19_start, today)

Correlations between stocks
  NOTE: Need to correlate on percentage change, not abosolute price


alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)